# Task10 BERT
'''
 
1. Transformer的原理。
2. BERT的原理。
3. 利用预训练的BERT模型将句子转换为句向量，进行文本分类。 

'''


In [ ]:
'''
3.利用预训练的BERT模型将句子转换为句向量，进行文本分类。

下载bert框架https://github.com/google-research/bert
修改run_classifier.py
下载预训练模型bert-base chinese版 chinese_L-12_H-768_A-12（已训练好，我们拿来在上面bert源码中加入我们要分类的文本即可直接分类）
run_classifier.py中加 DataProcessor子类MyClassifyProcessor，自己的处理数据集类
'''


In [ ]:
#==加我的分类数据集==
class CnnewsProcessor(DataProcessor):

  def __init__(self):
    self.labels = ['0', '1']#0为体育 1为财经
   #读取csv文件方法
  def _read_csv(self, input_file):
    with tf.gfile.Open(input_file, "r") as f:
        reader = csv.reader(f, delimiter=",", quotechar=None)
        lines = []
        for line in reader:
            lines.append(line)

    return lines

  def get_train_examples(self, data_dir):
    return self._create_examples(
      self._read_csv(os.path.join(data_dir, 'cnewsTrain.csv')), 'cnewsTrain')

  def get_dev_examples(self, data_dir):
    return self._create_examples(
      self._read_csv(os.path.join(data_dir, 'cnewsDev.csv')), 'cnewsDev')

  def get_test_examples(self, data_dir):
    return self._create_examples(
      self._read_csv(os.path.join(data_dir, 'cnewsTest.csv')), 'cnewsTest')

  def get_labels(self):
    return self.labels

  def _create_examples(self, lines, set_type):
    """create examples for the training and val sets"""
    examples = []
    for (i, line) in enumerate(lines):
      guid = '%s-%s' %(set_type, i)
      text_a = tokenization.convert_to_unicode(line[1])
      label = tokenization.convert_to_unicode(line[0])
      examples.append(InputExample(guid=guid, text_a=text_a, label=label))
    return examples

#====

In [ ]:

def main():
  processors = {
      "cola": ColaProcessor,
      "mnli": MnliProcessor,
      "mrpc": MrpcProcessor,
      "xnli": XnliProcessor,
      "cnnews":CnnewsProcessor
  }
#上面main 的processors中添加"MyClassify":MyClassifyProcessor



In [ ]:
'''
#命令行中执行以下语句，运行run_classifier.py
'''

#指定预训练模型路径(定义变量，给下面用。
#注意：window命令行用set设置，echo %BERT_BASE_DIR%可查看，使用用%变量名%，
#linux用export定义，使用用$变量名)

set BERT_BASE_DIR=G:/jt/tang/whale/bert/chinese_L-12_H-768_A-12
set DATA_DIR=G:/jt/tang/whale/bert/bert-classify/data
set OUTPUT_DIR = G:/jt/tang/whale/bert/

运行：

python G:/jt/tang/whale/bert/bert-classify/run_classifier.py \

 --task_name=cnnews \

 --do_train=true \

 --do_eval=true \

 --data_dir=%DATA_DIR%/ \

 --vocab_file=%BERT_BASE_DIR%/vocab.txt \

 --bert_config_file=%BERT_BASE_DIR%/bert_config.json \

 --init_checkpoint=%BERT_BASE_DIR%/bert_model.ckpt \

 --max_seq_length=128 \

 --train_batch_size=32 \

 --learning_rate=2e-5 \

 --num_train_epochs=3.0 \

 --output_dir=%OUTPUT_DIR%/tmp/cnnews_output

#/tmp/cnnews_output目录存放分类模型结果
#打开查看精确度
#以上是中文数据集


In [ ]:
#英文数据集-MRPC数据集分类
'''
下载谷歌预训练模型英文版 bert-base uncased
下载数据集MRPC
链接：https://pan.baidu.com/s/1-b4I3ocYhiuhu3bpSmCJ_Q
提取码：z6mk
数据与模型放入指定路径
'''
#不用修改，bert源码中自带MRPC类
#执行以下命令即可保存模型
set BERT_BASE_DIR=G:/jt/tang/whale/bert/en-uncased_L-12_H-768_A-12
set GLUE_DIR=G:/jt/tang/whale/bert/bert-mrpc/MRPC
set OUTPUT_DIR = G:/jt/tang/whale/bert/

python G:/jt/tang/whale/bert/bert-mrpc/run_classifier.py \
  --task_name=mrpc \
  --do_train=true \
  --do_eval=true \
  --data_dir=%GLUE_DIR% \
  --vocab_file=%BERT_BASE_DIR%/vocab.txt \
  --bert_config_file=%BERT_BASE_DIR%/bert_config.json \
  --init_checkpoint=%BERT_BASE_DIR%/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=%OUTPUT_DIR%/tmp/mrpc_output/

#tmp/mrpc_output/路径查看模型训练结果

In [ ]:
#预测
#命令行中执行以下语句
set BERT_BASE_DIR=G:/jt/tang/whale/bert/en-uncased_L-12_H-768_A-12
set GLUE_DIR=G:/jt/tang/whale/bert/bert-mrpc/MRPC
set OUTPUT_DIR = G:/jt/tang/whale/bert/
set TRAINED_CLASSIFIER=G:/jt/tang/whale/bert/bert-mrpc/MRPC/

python run_classifier.py \
  --task_name=mrpc \
  --do_predict=true \
  --data_dir=$DATA_DIR/\
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$TRAINED_CLASSIFIER \
  --max_seq_length=128 \
  --output_dir=/tmp/mrpc_output/


In [ ]:
'''
1. Transformer的原理。

Transformer的网络结构是对RNN的革新和之前的模型相差很大。
目前是一个非常有效且应用广泛的结构，是即seq2seq（sequence to sequnece）之后又一次“革命”。

先说Seq2seq（sequence to sequnece）：
它是输入一个序列（一串单元，比如词），根据关系权重，输出有关系的另一个序列（另一串单元，词），
结构是两个RNN，一个encoder编码器，一个decoder解码器。
以MT（机器翻译Machine translate）为例
输入：“我爱中国”输出“I love China”，是下面这样一个过程
输入X=(x_0,x_1,x_2,x_3) ，其中 x_0=“我” ， x_1=“爱” ， x_2=“中” ，x_2=“国” ；
定义目标序列（也就是target真实值）：T=(t_0,t_1,t_2)="I love China" ；
encoder，把 X=(x_0,x_1,x_2,x_3) 映射为一个隐层状态 h ，将 h 映射为 预测值 Y=(y_0,y_1,y_2)
最后将 预测值 Y与真实值T做loss（通常为交叉熵），训练网络。

Attention机制：
现在提到的seq2seq中通常都是带有Attention机制，
即指上面将隐层H映射到Y的时候不是单个词一一映射，而是每个词都两两相关带有相应权重的映射到Y.
每个元素通过不同的权值给与Y不同的贡献。
encoder编码器里是两个双向LSTM(一个记录处理中心词之前的词，一个处理之后的词)，
得到一个concat合并起来的H隐层。再向下传递。
Attention在计算权重的时候是自注意力，相当于给自己的每个词做了K-V组，
一个单元输入进来以后，去query查询匹配（做内积求相似度）整个句子中的其它词，得到一个相关度权重。

Transformer：
是对上面Seq2seq的一个升级改进，不同之处在于，
它的encoder和decoder不是用RNN循环网络的LSTM记忆单元之类的，
而是直接用了多个Attention,
这样做的好处是不用等隐层H中依次计算完传递再做解码。
而是可以并行计算，因为attention本身就是自注意力，本身就是词单元与其它单元之间的各自独立匹配，
不存在相互计算结果传递问题。
然后，这种self attention的任意两帧之间的直接依赖，
也好于RNN需要逐帧传递到最终的方式。
Transformer还有一点因为是Attention机制，帧之间不存在依赖关系，
所以要单独传一个地址编号（RNN是自带编号的），
所以它的embedding向量映射层包括两部分地址相关positional embedding和输入值input embedding
Attention还会连接一个Add norm 求和归一化 层，用于处理数据标准化，通过前馈神经网络feed forward向前传播
在解码器decoder中是同样结构，最后全连接层做线性转换连softmax
Attention机制将任意位置的两个单词的距离转换成1，有效的解决了NLP中棘手的长期依赖问题

参考：https://zhuanlan.zhihu.com/p/38485843
https://zhuanlan.zhihu.com/p/54356280

Transformer构架的核心思想是，
计算一句话中每个词对于这句话中其它所有词的相互关系，
然后认为这些词与词之间的相互关系
在一定程度上反应了这句话中不同词之间的关联性以及它们的重要程度。
再利用这些相互关系来调整每个词的重要性（权重）就可以获得每个词新的表达。
这个新的表征不但蕴含了该词本身，还蕴含了其他词与这个词的关系，
因此和单纯的词向量相比是一个更加全局的表达。
在翻译的应用中就可以用这个整体的关系去衡量另一种语言。

2. BERT的原理

ELMo：
是解决输入端nlp上游词向量化问题，针对文本输入的标准操作想要取代Word2vec，
改变没考虑上下文，输入一成不变的，没有表征一词多义的能力

GPT：
解决输出端NLP下游任务，提出了一个两阶段的模型。
第一阶段，在大规模的无监督文本上训练一个单向的Transformer模型；
第二阶段，只需要根据不同的任务要求，仅需要少量的特定标注数据进行调优训练，
即可获得若干任务数据集上STOA级别的表现。
这样的迁移学习方法，极大的节省了为不同的特定任务进行人工标注数据的高昂成本需求。

Bert:
Pre-training of Deep Bidirectional Transformers for Language Understanding
顺着GPT的思路，2018年10月BERT的横空出世，
将2018年NLP领域无监督学习的热潮推向了一个新的高度!
它在以下的几个方面在GPT的基础上有了进一步的突破：
Deep model,成功驾驭了一个
深达24层、每层1024个神经元、并且Multi-dead数为16、总参数340M的巨大Transformer模型
也体现了Transformer模型具有良好的稳定性这一重要优点，
因为其本身就含有各种Normalization。
相比于GPT，BERT在训练上解决了无法使用双向Transformer的窘境。
相比于ELMo，两个方向上的loss结合在一起，而不是相互独立，
更大程度上释放了Transformer构架的信息采集能力。
Jointly Pre-Train，针对NLP下游任务涉及到句子之间关系的理解，
在原有的loss函数后面新添加了一项关于句子间关系理解的loss，来进行联合训练。
将两个句子一起作为输入（中间有间隔符[SEP]）。
用无监督语料训练的时候，将一半的句子下一句进行随机句子采样，作为负例“NotNext”；
而其余一半则直接给出真正的后句，作为正例“IsNext”。
Position Embedding，BERT并没有采取用三角函数来表达句子中词语位置的方法，
而是直接设置句子的固定长度去训练Position Embedding，
在每个词的位置随机初始化词向量，经过训练，
将Position Embedding与Token Embedding、
以及模型训练得到的Segment Embeddings 直接相加即可食用。
feature-based,BERT不仅仅是一个Fine-tuning迁移学习方法，还可以是一个Feature-based迁移学习方法.
More Data,相比于GPT模型5GB、800M words 的训练数据，BERT更是用了惊人的3,200M words的训练数据。

之前的预训练模型（包括word2vec，ELMo等）都会生成词向量，属于domain transfer。
而近一两年提出的ULMFiT，GPT，BERT等都属于模型迁移。
BERT 模型是将预训练模型和下游任务模型结合在一起的，
也就是说在做下游任务时仍然是用BERT模型，而且天然支持文本分类任务，
在做文本分类任务时不需要对模型做修改。

参考：
https://www.jianshu.com/p/9f32d882321b
代码实践参考：
阅读理解等
http://www.pianshen.com/article/3178161910/
bert-service 输入词转向量
https://www.jianshu.com/p/bafdd927ef02
https://netycc.com/2018/12/05/%E5%88%A9%E7%94%A8bert%E6%9E%84%E5%BB%BA%E5%8F%A5%E5%90%91%E9%87%8F%E5%B9%B6%E8%AE%A1%E7%AE%97%E7%9B%B8%E4%BC%BC%E5%BA%A6/

'''
